In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
rows = x.shape[0] 
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 1ms/step


In [2]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
gaussian_nb = GaussianNB()
gaussian_nb.fit(x_train, y_train)
y_pred_gaussian = gaussian_nb.predict(x_test)

In [4]:
print("Gaussian Naive Bayes:")
print("Accuracy:", accuracy_score(y_test, y_pred_gaussian))
print("Classification Report:\n", classification_report(y_test, y_pred_gaussian))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gaussian))
print("-" * 30)

Gaussian Naive Bayes:
Accuracy: 0.468975468975469
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.27      0.37       232
           1       0.61      0.27      0.37       231
           2       0.42      0.87      0.56       230

    accuracy                           0.47       693
   macro avg       0.53      0.47      0.43       693
weighted avg       0.53      0.47      0.43       693

Confusion Matrix:
 [[ 63  30 139]
 [ 30  62 139]
 [ 20  10 200]]
------------------------------


In [5]:
bernoulli_nb = BernoulliNB()
bernoulli_nb.fit(x_train > 0, y_train) # Treat non-zero as present (1), zero as absent (0)
y_pred_bernoulli = bernoulli_nb.predict(x_test > 0)

print("Bernoulli Naive Bayes:")
print("Accuracy:", accuracy_score(y_test, y_pred_bernoulli))
print("Classification Report:\n", classification_report(y_test, y_pred_bernoulli))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bernoulli))
print("-" * 30)

Bernoulli Naive Bayes:
Accuracy: 0.5425685425685426
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.41      0.54       232
           1       0.58      0.41      0.48       231
           2       0.45      0.81      0.58       230

    accuracy                           0.54       693
   macro avg       0.61      0.54      0.53       693
weighted avg       0.61      0.54      0.53       693

Confusion Matrix:
 [[ 94  45  93]
 [  5  95 131]
 [ 20  23 187]]
------------------------------
